##Test Problem

In [ ]:
import urllib.request
import numpy as np
from itertools import combinations

# Scarica il file .npy da GitHub
url = 'https://raw.githubusercontent.com/squillero/computational-intelligence/master/2025-26/lab2/problem_g_500.npy'
urllib.request.urlretrieve(url, 'problem_g_500.npy')

# Carica la matrice
problem = np.load('problem_g_500.npy')

In [ ]:
# Test: Valori negativi?
print("Contiene valori negativi?", np.any(problem < 0))

# Test: Diagonale tutta zero?
print("Diagonale tutta zero?", np.allclose(np.diag(problem), 0.0))

# Test: Matrice simmetrica?
print("Matrice simmetrica?", np.allclose(problem, problem.T))

# Test: Disuguaglianza triangolare
triangle_ok = all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in combinations(range(problem.shape[0]), 3)
)
print("Rispetta la disuguaglianza triangolare?", triangle_ok)


Contiene valori negativi? False
Diagonale tutta zero? True
Matrice simmetrica? True


KeyboardInterrupt: 

##Only crossover and mutation without local search
##There are all most important crossover, we have to choose best for this task
##In mutation, there is not scrumble mutation

In [ ]:
import numpy as np
import random
from typing import List, Tuple, Callable


# ---------------------------
# Helper: valutazione fitness
# ---------------------------

def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """Calcola la lunghezza totale di un tour (chiuso) dato un indice di città."""
    # somma distanze tra città consecutive + ritorno alla città iniziale
    n = len(tour)
    # dist_matrix[tour[i], tour[i+1]] per i=0..n-2 e infine tour[-1] -> tour[0]
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# -----------------------------------------
# Inizializzazione: Nearest Neighbor (NN)
# -----------------------------------------

def nearest_neighbor_tour(start: int, dist_matrix: np.ndarray) -> np.ndarray:
    """Costruisce un tour con euristica nearest neighbor partendo da 'start'."""
    n = dist_matrix.shape[0]
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    current = start

    while unvisited:
        # scegli la città non visitata più vicina
        next_city = min(unvisited, key=lambda j: dist_matrix[current, j])
        tour.append(next_city)
        unvisited.remove(next_city)
        current = next_city

    return np.array(tour, dtype=int)


def init_population_nn(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """Inizializza la popolazione usando NN con diversi starting points.
    Se pop_size > n start da ogni città più alcune varianti randomizzate."""
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)

    n = dist_matrix.shape[0]
    population = []

    # usiamo tutti gli start possibili finché necessario
    starts = list(range(n))
    idx = 0

    while len(population) < pop_size:
        start = starts[idx % n]
        base = nearest_neighbor_tour(start, dist_matrix)
        population.append(base)
        idx += 1

        # se abbiamo già usato tutti gli start ma serve ancora popolazione,
        # generiamo varianti dell'NN permutando tie-breaker: small random swap
        if idx >= n and len(population) < pop_size:
            # prendi un NN casuale e applica piccole perturbazioni per varietà
            b = nearest_neighbor_tour(random.choice(starts), dist_matrix).copy()
            # applica 1-3 swap a caso
            for _ in range(random.randint(1, 3)):
                i, j = random.sample(range(n), 2)
                b[i], b[j] = b[j], b[i]
            population.append(b)

    return population[:pop_size]


# -------------------------
# Operatori di Crossover
# -------------------------

def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """OX - Order Crossover (restituisce due figli)."""
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    # copia segmento
    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    # riempi il resto in ordine rispetto all'altro genitore
    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)


def partially_mapped_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """PMX - Partially Mapped Crossover."""
    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))
    child1 = np.full(n, -1, dtype=int)
    child2 = np.full(n, -1, dtype=int)

    # copia la porzione centrale
    child1[a:b + 1] = parent1[a:b + 1]
    child2[a:b + 1] = parent2[a:b + 1]

    # mappatura e inserimento
    def pmx_fill(child, donor_segment, donor_full):
        a_local, b_local = a, b
        for i in list(range(0, a_local)) + list(range(b_local + 1, n)):
            gene = donor_full[i]
            while gene in donor_segment:
                idx_in_segment = np.where(donor_segment == gene)[0][0]
                gene = donor_full[a_local + idx_in_segment]
            child[i] = gene
        return child

    child1 = pmx_fill(child1, parent1[a:b + 1], parent2)
    child2 = pmx_fill(child2, parent2[a:b + 1], parent1)

    return child1, child2


def cycle_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """CX - Cycle Crossover."""
    n = len(parent1)
    child1 = np.full(n, -1, dtype=int)
    child2 = np.full(n, -1, dtype=int)
    cycles = np.full(n, -1, dtype=int)
    cycle_id = 0

    for i in range(n):
        if cycles[i] != -1:
            continue
        idx = i
        while cycles[idx] == -1:
            cycles[idx] = cycle_id
            val = parent2[idx]
            idx = int(np.where(parent1 == val)[0][0])
        cycle_id += 1

    for pos in range(n):
        if cycles[pos] % 2 == 0:
            child1[pos] = parent1[pos]
            child2[pos] = parent2[pos]
        else:
            child1[pos] = parent2[pos]
            child2[pos] = parent1[pos]

    return child1, child2


  # ---- ERX: Edge Recombination Crossover --------------------------------------

def edge_recombination_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """ERX - Edge Recombination Crossover.
    Preserva adiacenze dei genitori costruendo un grafo di adiacenza (edge map)
    e scegliendo il prossimo gene tra i vicini del corrente con regola del grado minimo.
    Restituisce due figli, con start diversi (p1[0] e p2[0])."""

    def build_adjacency(p1: np.ndarray, p2: np.ndarray):
        n = len(p1)
        adj = {int(c): set() for c in p1.tolist()}

        def add_edges(p: np.ndarray):
            for i in range(n):
                c = int(p[i])
                left = int(p[(i - 1) % n])
                right = int(p[(i + 1) % n])
                adj[c].add(left)
                adj[c].add(right)

        add_edges(p1)
        add_edges(p2)
        return adj

    def erx_one(p1: np.ndarray, p2: np.ndarray, start_city: int = None) -> np.ndarray:
        n = len(p1)
        adj = build_adjacency(p1, p2)
        child = np.full(n, -1, dtype=int)
        unvisited = set(int(x) for x in p1.tolist())

        current = int(p1[0]) if start_city is None else int(start_city)
        for i in range(n):
            child[i] = current
            unvisited.remove(current)

            # rimuovi 'current' da tutte le liste di adiacenza
            for s in adj.values():
                s.discard(current)

            # scegli il prossimo tra i vicini ancora non visitati (regola del grado minimo)
            candidates = [c for c in adj[current] if c in unvisited]
            if candidates:
                min_deg = min(len(adj[c]) for c in candidates)
                candidates = [c for c in candidates if len(adj[c]) == min_deg]
                current = random.choice(candidates)
            else:
                # nessun vicino disponibile: scegli un nodo qualsiasi non visitato
                if unvisited:
                    current = random.choice(list(unvisited))
                else:
                    current = None  # finito

        return child

    # Due figli con start diversi; invertiamo l'ordine dei genitori nel secondo
    c1 = erx_one(parent1, parent2, start_city=int(parent1[0]))
    c2 = erx_one(parent2, parent1, start_city=int(parent2[0]))
    return c1, c2


# -------------------------
# Operator di Mutazione
# -------------------------

def swap_mutation(ind: np.ndarray, n_swaps: int = 1) -> np.ndarray:
    """Swap mutation: scambia n_swaps coppie casuali."""
    ind = ind.copy()
    n = len(ind)
    for _ in range(n_swaps):
        i, j = random.sample(range(n), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


def inversion_mutation(ind: np.ndarray) -> np.ndarray:
    """Inversion mutation: seleziona un sotto-intervallo e lo inverte."""
    ind = ind.copy()
    n = len(ind)
    a, b = sorted(random.sample(range(n), 2))
    ind[a:b + 1] = ind[a:b + 1][::-1]
    return ind


# -------------------------
# Selezione: torneo + elitismo
# -------------------------

def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    """Selezione torneo di dimensione k (ritorna un clone dell'individuo scelto)."""
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# -------------------------
# Algoritmo Evolutivo main
# -------------------------

def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:
    """Algoritmo evolutivo per TSP con NN init, OX/PMX/CX, swap/inversion mutations."""
    random.seed(rng_seed)
    np.random.seed(rng_seed)

    n = dist_matrix.shape[0]

    # Inizializzazione popolazione usando NN
    population = init_population_nn(pop_size, dist_matrix, rng_seed=rng_seed)

    # Valuta fitness iniziale
    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    # operatori disponibili
    crossover_ops: List[Callable[[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]] = [
        order_crossover,
        partially_mapped_crossover,
        cycle_crossover,
        edge_recombination_crossover
    ]
    mutation_ops: List[Callable[[np.ndarray], np.ndarray]] = [
        swap_mutation,
        inversion_mutation
    ]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        # ordina popolazione per fitness (minimizzazione)
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # aggiorna il migliore globale
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        # crea next generation
        new_population = []

        # mantieni elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # riempi il resto della popolazione
        while len(new_population) < pop_size:
            # seleziona genitori con torneo
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)

            # previeni identità triviale
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # crossover probabilistico
            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                child1, child2 = cx(parent1, parent2)
            else:
                # clonazione
                child1, child2 = parent1.copy(), parent2.copy()

            # mutazione probabilistica
            if random.random() < mutation_rate:
                mut = random.choice(mutation_ops)
                child1 = mut(child1)
            if random.random() < mutation_rate:
                mut = random.choice(mutation_ops)
                child2 = mut(child2)

            # aggiungi figli alla nuova popolazione
            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        # aggiorna popolazione e fitness
        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        # stampa di progresso
        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# -------------------------
# Esecuzione
# -------------------------

if __name__ == "__main__":

    # parametri di esempio — regola a piacere
    POP_SIZE = 80
    GENERATIONS = 5000
    CROSSOVER_RATE = 0.95
    MUTATION_RATE = 0.3
    ELITISM = 4
    TOURN_K = 4
    SEED = 1234

    best_tour, best_cost = evolutionary_tsp(
        problem,
        pop_size=POP_SIZE,
        generations=GENERATIONS,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE,
        elitism_count=ELITISM,
        tournament_k=TOURN_K,
        rng_seed=SEED,
    )

    print("\n--- Risultato finale ---")
    print("Costo migliore:", best_cost)
    print("Tour migliore (ordine città):", best_tour)


Gen    1 | Best so far: 9658.6731
Gen  500 | Best so far: 9619.0570


KeyboardInterrupt: 

##2-OPT for standard problem
##it's a local search after applying evolution, it's the best techinique for standard problem but really slow. it can't be use for asymmetric problem (the next ones)

In [ ]:
import numpy as np
import random
from typing import List, Tuple, Callable


# ---------------------------
# Helper: valutazione fitness
# ---------------------------

def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """Calcola la lunghezza totale di un tour (chiuso) dato un indice di città."""
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# -----------------------------------------
# Inizializzazione: Random pura
# -----------------------------------------

def init_population_random(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """Inizializza la popolazione con tour generati casualmente."""
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)

    n = dist_matrix.shape[0]
    population = [np.random.permutation(n) for _ in range(pop_size)]
    return population


# -------------------------
# Operatori di Crossover
# -------------------------

def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """OX - Order Crossover (restituisce due figli)."""
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    # copia segmento
    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    # riempi il resto
    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)


# -------------------------
# Mutazione
# -------------------------

def inversion_mutation(ind: np.ndarray) -> np.ndarray:
    """Inversion mutation: seleziona un sotto-intervallo e lo inverte."""
    ind = ind.copy()
    n = len(ind)
    a, b = sorted(random.sample(range(n), 2))
    ind[a:b + 1] = ind[a:b + 1][::-1]
    return ind


# -------------------------
# Local Search: 2-OPT
# -------------------------

def two_opt(tour: np.ndarray, dist_matrix: np.ndarray) -> np.ndarray:
    """Migliora un tour applicando mosse 2-OPT finché trova miglioramenti."""
    n = len(tour)
    tour = tour.copy()
    improved = True

    while improved:
        improved = False
        for i in range(n - 1):
            a, b = tour[i], tour[(i + 1) % n]
            k_max = n if i > 0 else n - 1
            for k in range(i + 2, k_max):
                c, d = tour[k], tour[(k + 1) % n]
                delta = (dist_matrix[a, c] + dist_matrix[b, d]) - (dist_matrix[a, b] + dist_matrix[c, d])
                if delta < -1e-12:
                    tour[i + 1:k + 1] = tour[i + 1:k + 1][::-1]
                    improved = True
                    break
            if improved:
                break
    return tour


# -------------------------
# Selezione torneo
# -------------------------

def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# -------------------------
# Algoritmo Evolutivo
# -------------------------

def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    local_search_rate: float = 0.3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:

    random.seed(rng_seed)
    np.random.seed(rng_seed)

    n = dist_matrix.shape[0]

    # ✅ Inizializzazione casuale
    population = init_population_random(pop_size, dist_matrix, rng_seed=rng_seed)

    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    crossover_ops = [order_crossover]
    mutation_ops = [inversion_mutation]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        new_population = []

        # elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # generazione successiva
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                child1, child2 = cx(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if random.random() < mutation_rate:
                child1 = random.choice(mutation_ops)(child1)
            if random.random() < mutation_rate:
                child2 = random.choice(mutation_ops)(child2)

            # Local search 2-OPT (memetico)
            if random.random() < local_search_rate:
                child1 = two_opt(child1, dist_matrix)
            if random.random() < local_search_rate:
                child2 = two_opt(child2, dist_matrix)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# -------------------------
# Esempio di esecuzione
# -------------------------
if __name__ == "__main__":

    best_tour, best_cost = evolutionary_tsp(
        problem,
        pop_size=10,
        generations=10,
        crossover_rate=0.95,
        mutation_rate=0.3,
        elitism_count=1,
        tournament_k=1,
        local_search_rate=0.3,
        rng_seed=1234,
    )

    print("\n--- Risultato finale ---")
    print("Costo migliore:", best_cost)
    print("Tour migliore:", best_tour)


KeyboardInterrupt: 

##Risoluzione problema ATSP

In [ ]:
import numpy as np
import random
from typing import List, Tuple


# ==========================================================
#  FUNZIONE DI FITNESS
# ==========================================================
def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """
    Calcola la lunghezza totale di un tour chiuso (TSP).
    """
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# ==========================================================
#  INIZIALIZZAZIONE POPOLAZIONE
# ==========================================================
def init_population_random(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """
    Crea una popolazione iniziale casuale di tour (permutazioni di città).
    """
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)

    n = dist_matrix.shape[0]
    population = [np.random.permutation(n) for _ in range(pop_size)]
    return population


# ==========================================================
#  OPERATORI DI CROSSOVER
# ==========================================================
def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    OX (Order Crossover): mantiene l’ordine relativo delle città.
    Restituisce due figli.
    """
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    # copia il segmento
    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    # riempie i restanti con l'ordine del genitore opposto
    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)


# ==========================================================
#  OPERATORI DI MUTAZIONE
# ==========================================================

def swap_mutation(ind: np.ndarray) -> np.ndarray:
    """Scambia due città a caso nel tour."""
    ind = ind.copy()
    a, b = random.sample(range(len(ind)), 2)
    ind[a], ind[b] = ind[b], ind[a]
    return ind


def scramble_mutation(ind: np.ndarray) -> np.ndarray:
    """Rimescola casualmente un sottoinsieme di città."""
    ind = ind.copy()
    n = len(ind)
    a, b = sorted(random.sample(range(n), 2))
    subset = ind[a:b + 1]
    np.random.shuffle(subset)
    ind[a:b + 1] = subset
    return ind


# ==========================================================
#  SELEZIONE TORNEO
# ==========================================================
def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    """
    Seleziona il miglior individuo tra k scelti casualmente.
    """
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# ==========================================================
#  ALGORITMO EVOLUTIVO
# ==========================================================
def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:
    """
    Algoritmo evolutivo per il TSP.
    - Popolazione iniziale casuale
    - Selezione per torneo
    - Crossover OX
    - Mutazioni: inversione, scambio, rimescolamento, inserimento
    - Elitismo per mantenere i migliori individui
    """
    random.seed(rng_seed)
    np.random.seed(rng_seed)

    # Inizializzazione popolazione
    population = init_population_random(pop_size, dist_matrix, rng_seed)
    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    crossover_ops = [order_crossover]
    mutation_ops = [swap_mutation, scramble_mutation]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        # Ordina per fitness
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # Aggiorna miglior individuo
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        new_population = []

        # Elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # Genera nuova popolazione
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # Crossover
            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                child1, child2 = cx(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Mutazione
            if random.random() < mutation_rate:
                child1 = random.choice(mutation_ops)(child1)
            if random.random() < mutation_rate:
                child2 = random.choice(mutation_ops)(child2)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        # Stampa progresso
        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# ==========================================================
#  ESEMPIO DI ESECUZIONE
# ==========================================================
if __name__ == "__main__":

    best_tour, best_cost = evolutionary_tsp(
        dist_matrix=problem,
        pop_size=50,
        generations=1000,
        crossover_rate=0.95,
        mutation_rate=0.3,
        elitism_count=2,
        tournament_k=3,
        rng_seed=1234,
    )

    print("\n--- RISULTATO FINALE ---")
    print("Miglior costo:", best_cost)
    print("Miglior tour:", best_tour)


Gen    1 | Best so far: 4777.0701
Gen  100 | Best so far: 2291.7745
Gen  200 | Best so far: 1806.3930
Gen  300 | Best so far: 1572.2121
Gen  400 | Best so far: 1395.1720
Gen  500 | Best so far: 1322.6141
Gen  600 | Best so far: 1215.3452
Gen  700 | Best so far: 1211.2335
Gen  800 | Best so far: 1180.2480
Gen  900 | Best so far: 1092.6014
Gen 1000 | Best so far: 1075.3023

--- RISULTATO FINALE ---
Miglior costo: 1075.3022981481686
Miglior tour: [ 5 22 33 63  9 16 85 71 75 13 45 12 48 24 57 90 47 23 19 41 60 44 89 21
  7 27 62 28 64 96  2 11  8 99 73 52 29 40 30 86 14 35 79 34 18 91 42  0
  1 76 59 61 92 10 68 15 88 97 37 82 39  3  6 67 66 31 72 32 54 98 49 51
 80 43 17 84 87 93 74 95 69 26 46 77 65 55 83 20 25 70 56 81 50 78 38 94
 58  4 53 36]


##Risoluzione ATSP con buon crossover

In [ ]:
import numpy as np
import random
from typing import List, Tuple


# ==========================================================
#  FUNZIONE DI FITNESS
# ==========================================================
def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """Calcola la lunghezza totale di un tour chiuso (TSP o ATSP)."""
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# ==========================================================
#  INIZIALIZZAZIONE POPOLAZIONE
# ==========================================================
def init_population_random(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """Crea una popolazione iniziale casuale di tour (permutazioni di città)."""
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)
    n = dist_matrix.shape[0]
    return [np.random.permutation(n) for _ in range(pop_size)]


# ==========================================================
#  OPERATORI DI CROSSOVER
# ==========================================================
def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    OX (Order Crossover): mantiene l’ordine relativo delle città.
    Restituisce due figli.
    """
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)


def scx_crossover(parent1: np.ndarray, parent2: np.ndarray, dist_matrix: np.ndarray) -> np.ndarray:
    """
    SCX (Sequential Constructive Crossover) — adatto per ATSP.
    Costruisce il figlio scegliendo la prossima città in base al costo diretto.
    """
    n = len(parent1)
    child = [parent1[0]]
    remaining = set(range(n))
    remaining.remove(parent1[0])

    while remaining:
        current = child[-1]
        # Trova le prossime città nei genitori
        idx1 = np.where(parent1 == current)[0][0]
        idx2 = np.where(parent2 == current)[0][0]
        next1 = parent1[(idx1 + 1) % n]
        next2 = parent2[(idx2 + 1) % n]

        # Candidati non ancora visitati
        candidates = []
        if next1 in remaining:
            candidates.append(next1)
        if next2 in remaining and next2 != next1:
            candidates.append(next2)

        # Scegli la migliore candidata
        if candidates:
            next_city = min(candidates, key=lambda c: dist_matrix[current, c])
        else:
            # Tutti visitati → scegli la città rimanente più vicina
            next_city = min(remaining, key=lambda c: dist_matrix[current, c])

        child.append(next_city)
        remaining.remove(next_city)

    return np.array(child)


# ==========================================================
#  OPERATORI DI MUTAZIONE
# ==========================================================
def swap_mutation(ind: np.ndarray) -> np.ndarray:
    """Scambia due città a caso nel tour."""
    ind = ind.copy()
    a, b = random.sample(range(len(ind)), 2)
    ind[a], ind[b] = ind[b], ind[a]
    return ind


def scramble_mutation(ind: np.ndarray) -> np.ndarray:
    """Rimescola casualmente un sottoinsieme di città."""
    ind = ind.copy()
    n = len(ind)
    a, b = sorted(random.sample(range(n), 2))
    subset = ind[a:b + 1]
    np.random.shuffle(subset)
    ind[a:b + 1] = subset
    return ind


# ==========================================================
#  SELEZIONE TORNEO
# ==========================================================
def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    """Seleziona il miglior individuo tra k scelti casualmente."""
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# ==========================================================
#  ALGORITMO EVOLUTIVO
# ==========================================================
def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:
    """
    Algoritmo evolutivo per TSP/ATSP con crossover SCX.
    """
    random.seed(rng_seed)
    np.random.seed(rng_seed)

    # Inizializzazione popolazione
    population = init_population_random(pop_size, dist_matrix, rng_seed)
    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    # Aggiungiamo SCX ai crossover disponibili
    crossover_ops = [scx_crossover]
    mutation_ops = [swap_mutation, scramble_mutation]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        # Ordina per fitness
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # Aggiorna miglior individuo
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        new_population = []

        # Elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # Genera nuova popolazione
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # Crossover
            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                if cx == scx_crossover:
                    child1 = scx_crossover(parent1, parent2, dist_matrix)
                    child2 = scx_crossover(parent2, parent1, dist_matrix)
                else:
                    child1, child2 = cx(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Mutazione
            if random.random() < mutation_rate:
                child1 = random.choice(mutation_ops)(child1)
            if random.random() < mutation_rate:
                child2 = random.choice(mutation_ops)(child2)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        # Stampa progresso
        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# ==========================================================
#  ESEMPIO DI ESECUZIONE
# ==========================================================
if __name__ == "__main__":
    # Carica la matrice ATSP (già definita altrove)
    # problem = np.load('test_problem.npy')

    best_tour, best_cost = evolutionary_tsp(
        dist_matrix=problem,
        pop_size=50,
        generations=1000,
        crossover_rate=0.95,
        mutation_rate=0.3,
        elitism_count=2,
        tournament_k=3,
        rng_seed=1234,
    )

    print("\n--- RISULTATO FINALE ---")
    print("Miglior costo:", best_cost)
    print("Miglior tour:", best_tour)


Gen    1 | Best so far: 4777.0701
Gen  100 | Best so far: 792.3947
Gen  200 | Best so far: 792.3947
Gen  300 | Best so far: 768.1187
Gen  400 | Best so far: 768.1187
Gen  500 | Best so far: 768.1187
Gen  600 | Best so far: 768.1187
Gen  700 | Best so far: 768.1187
Gen  800 | Best so far: 768.1187
Gen  900 | Best so far: 768.1187
Gen 1000 | Best so far: 768.1187

--- RISULTATO FINALE ---
Miglior costo: 768.1186714416355
Miglior tour: [14 64 60 37 30 86 40 56 25 70 84 17 43 41 78 53  5 22 80 94 51 36 58 88
 15 79  2 50 81 35  8 99 73 29 52 74 93 95 69 26 46 77 20 83 65 55 87 47
 23 19 45 90 48 24 57 12  4 13 75 85 71  7 27 97 42 18 82 72 32 16 98 54
 49 38 31  9 63 33 39 66  3  6 67 59 61 92 76  1  0 91 89 44 21 34 62 28
 68 10 11 96]


In [ ]:
import numpy as np
import random
from typing import List, Tuple, Callable


# ---------------------------
# Helper: valutazione fitness
# ---------------------------

def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """Calcola la lunghezza totale di un tour (chiuso) dato un indice di città."""
    # somma distanze tra città consecutive + ritorno alla città iniziale
    n = len(tour)
    # dist_matrix[tour[i], tour[i+1]] per i=0..n-2 e infine tour[-1] -> tour[0]
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# -----------------------------------------
# Inizializzazione: Nearest Neighbor (NN)
# -----------------------------------------

def nearest_neighbor_tour(start: int, dist_matrix: np.ndarray) -> np.ndarray:
    """Costruisce un tour con euristica nearest neighbor partendo da 'start'."""
    n = dist_matrix.shape[0]
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    current = start

    while unvisited:
        # scegli la città non visitata più vicina
        next_city = min(unvisited, key=lambda j: dist_matrix[current, j])
        tour.append(next_city)
        unvisited.remove(next_city)
        current = next_city

    return np.array(tour, dtype=int)


def init_population_nn(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """Inizializza la popolazione usando NN con diversi starting points.
    Se pop_size > n start da ogni città più alcune varianti randomizzate."""
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)

    n = dist_matrix.shape[0]
    population = []

    # usiamo tutti gli start possibili finché necessario
    starts = list(range(n))
    idx = 0

    while len(population) < pop_size:
        start = starts[idx % n]
        base = nearest_neighbor_tour(start, dist_matrix)
        population.append(base)
        idx += 1

        # se abbiamo già usato tutti gli start ma serve ancora popolazione,
        # generiamo varianti dell'NN permutando tie-breaker: small random swap
        if idx >= n and len(population) < pop_size:
            # prendi un NN casuale e applica piccole perturbazioni per varietà
            b = nearest_neighbor_tour(random.choice(starts), dist_matrix).copy()
            # applica 1-3 swap a caso
            for _ in range(random.randint(1, 3)):
                i, j = random.sample(range(n), 2)
                b[i], b[j] = b[j], b[i]
            population.append(b)

    return population[:pop_size]


# -------------------------
# Operatori di Crossover
# -------------------------

def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """OX - Order Crossover (restituisce due figli)."""
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    # copia segmento
    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    # riempi il resto in ordine rispetto all'altro genitore
    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)

# -------------------------
# Operator di Mutazione
# -------------------------

def swap_mutation(ind: np.ndarray, n_swaps: int = 1) -> np.ndarray:
    """Swap mutation: scambia n_swaps coppie casuali."""
    ind = ind.copy()
    n = len(ind)
    for _ in range(n_swaps):
        i, j = random.sample(range(n), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind

# -------------------------
# Selezione: torneo + elitismo
# -------------------------

def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    """Selezione torneo di dimensione k (ritorna un clone dell'individuo scelto)."""
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# -------------------------
# Algoritmo Evolutivo main
# -------------------------

def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:
    """Algoritmo evolutivo per TSP con NN init, OX/PMX/CX, swap/inversion mutations."""
    random.seed(rng_seed)
    np.random.seed(rng_seed)

    n = dist_matrix.shape[0]

    # Inizializzazione popolazione usando NN
    population = init_population_nn(pop_size, dist_matrix, rng_seed=rng_seed)

    # Valuta fitness iniziale
    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    # operatori disponibili
    crossover_ops: List[Callable[[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]] = [
        order_crossover
    ]
    mutation_ops: List[Callable[[np.ndarray], np.ndarray]] = [
        swap_mutation
    ]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        # ordina popolazione per fitness (minimizzazione)
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # aggiorna il migliore globale
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        # crea next generation
        new_population = []

        # mantieni elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # riempi il resto della popolazione
        while len(new_population) < pop_size:
            # seleziona genitori con torneo
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)

            # previeni identità triviale
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # crossover probabilistico
            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                child1, child2 = cx(parent1, parent2)
            else:
                # clonazione
                child1, child2 = parent1.copy(), parent2.copy()

            # mutazione probabilistica
            if random.random() < mutation_rate:
                mut = random.choice(mutation_ops)
                child1 = mut(child1)
            if random.random() < mutation_rate:
                mut = random.choice(mutation_ops)
                child2 = mut(child2)

            # aggiungi figli alla nuova popolazione
            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        # aggiorna popolazione e fitness
        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        # stampa di progresso
        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# -------------------------
# Esecuzione
# -------------------------

if __name__ == "__main__":

    # parametri di esempio — regola a piacere
    POP_SIZE = 80
    GENERATIONS = 5000
    CROSSOVER_RATE = 0.95
    MUTATION_RATE = 0.3
    ELITISM = 4
    TOURN_K = 4
    SEED = 1234

    best_tour, best_cost = evolutionary_tsp(
        problem,
        pop_size=POP_SIZE,
        generations=GENERATIONS,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE,
        elitism_count=ELITISM,
        tournament_k=TOURN_K,
        rng_seed=SEED,
    )

    print("\n--- Risultato finale ---")
    print("Costo migliore:", best_cost)
    print("Tour migliore (ordine città):", best_tour)


Gen    1 | Best so far: 9658.6731
Gen  500 | Best so far: 9481.6375
Gen 1000 | Best so far: 9475.3407
Gen 1500 | Best so far: 9475.2318
Gen 2000 | Best so far: 9463.0630
Gen 2500 | Best so far: 9463.0630
Gen 3000 | Best so far: 9451.3432
Gen 3500 | Best so far: 9444.9031
Gen 4000 | Best so far: 9443.8313
Gen 4500 | Best so far: 9426.8726
Gen 5000 | Best so far: 9382.7636

--- Risultato finale ---
Costo migliore: 9382.763594446547
Tour migliore (ordine città): [356 358 211 255 460 436 479 397 349 247 281 353   8 109 174  72 271 493
 140 221 490 426 417 143 122 344 203 121  14 215 316 467  74 428 449 182
 136 131 357 139  73 486 167 430 217 110  93 102 435 301 115 287 120 307
  27  95  61  21 235   3 116  36  65 454 403 315  94 277 172 274 202 199
  98 481  62 340 253 333 495  56 352 108   0 456 312 244 145 190 177  44
 165 123 477  55 374 222  78 472 365  53  28  38  79  39 377 286 194  99
  64 134 186 150  17  40 310  43  75  70 431 297 111  71 105 198  63 476
 233 160 415 127 104   4 

##Risoluzione ATSP non metric

In [ ]:
import numpy as np
import random
from typing import List, Tuple


# ==========================================================
#  FUNZIONE DI FITNESS
# ==========================================================
def tour_length(tour: np.ndarray, dist_matrix: np.ndarray) -> float:
    """
    Calcola la lunghezza totale di un tour chiuso (TSP).
    """
    return float(np.sum(dist_matrix[tour, np.roll(tour, -1)]))


# ==========================================================
#  INIZIALIZZAZIONE POPOLAZIONE
# ==========================================================
def init_population_random(pop_size: int, dist_matrix: np.ndarray, rng_seed: int = None) -> List[np.ndarray]:
    """
    Crea una popolazione iniziale casuale di tour (permutazioni di città).
    """
    if rng_seed is not None:
        random.seed(rng_seed)
        np.random.seed(rng_seed)

    n = dist_matrix.shape[0]
    population = [np.random.permutation(n) for _ in range(pop_size)]
    return population


# ==========================================================
#  OPERATORI DI CROSSOVER
# ==========================================================
def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    OX (Order Crossover): mantiene l’ordine relativo delle città.
    Restituisce due figli.
    """
    n = len(parent1)
    c1, c2 = np.full(n, -1, dtype=int), np.full(n, -1, dtype=int)
    a, b = sorted(random.sample(range(n), 2))

    # copia il segmento
    c1[a:b + 1] = parent1[a:b + 1]
    c2[a:b + 1] = parent2[a:b + 1]

    # riempie i restanti con l'ordine del genitore opposto
    def fill(child, donor):
        pos = (b + 1) % n
        for gene in np.concatenate((donor[b + 1:], donor[:b + 1])):
            if gene not in child:
                child[pos] = gene
                pos = (pos + 1) % n
        return child

    return fill(c1, parent2), fill(c2, parent1)


# ==========================================================
#  OPERATORI DI MUTAZIONE
# ==========================================================

def swap_mutation(ind: np.ndarray) -> np.ndarray:
    """Scambia due città a caso nel tour."""
    ind = ind.copy()
    a, b = random.sample(range(len(ind)), 2)
    ind[a], ind[b] = ind[b], ind[a]
    return ind


def scramble_mutation(ind: np.ndarray) -> np.ndarray:
    """Rimescola casualmente un sottoinsieme di città."""
    ind = ind.copy()
    n = len(ind)
    a, b = sorted(random.sample(range(n), 2))
    subset = ind[a:b + 1]
    np.random.shuffle(subset)
    ind[a:b + 1] = subset
    return ind


# ==========================================================
#  SELEZIONE TORNEO
# ==========================================================
def tournament_selection(population: List[np.ndarray], fitnesses: List[float], k: int) -> np.ndarray:
    """
    Seleziona il miglior individuo tra k scelti casualmente.
    """
    selected_idx = random.sample(range(len(population)), k)
    best = min(selected_idx, key=lambda i: fitnesses[i])
    return population[best].copy()


# ==========================================================
#  ALGORITMO EVOLUTIVO
# ==========================================================
def evolutionary_tsp(
    dist_matrix: np.ndarray,
    pop_size: int = 100,
    generations: int = 500,
    crossover_rate: float = 0.9,
    mutation_rate: float = 0.2,
    elitism_count: int = 2,
    tournament_k: int = 3,
    rng_seed: int = 42,
) -> Tuple[np.ndarray, float]:
    """
    Algoritmo evolutivo per il TSP.
    - Popolazione iniziale casuale
    - Selezione per torneo
    - Crossover OX
    - Mutazioni: inversione, scambio, rimescolamento, inserimento
    - Elitismo per mantenere i migliori individui
    """
    random.seed(rng_seed)
    np.random.seed(rng_seed)

    # Inizializzazione popolazione
    population = init_population_random(pop_size, dist_matrix, rng_seed)
    fitnesses = [tour_length(ind, dist_matrix) for ind in population]

    crossover_ops = [order_crossover]
    mutation_ops = [swap_mutation, scramble_mutation]

    best_overall = None
    best_overall_fit = float('inf')

    for gen in range(1, generations + 1):
        # Ordina per fitness
        sorted_idx = sorted(range(len(population)), key=lambda i: fitnesses[i])
        population = [population[i] for i in sorted_idx]
        fitnesses = [fitnesses[i] for i in sorted_idx]

        # Aggiorna miglior individuo
        if fitnesses[0] < best_overall_fit:
            best_overall_fit = fitnesses[0]
            best_overall = population[0].copy()

        new_population = []

        # Elitismo
        for i in range(elitism_count):
            new_population.append(population[i].copy())

        # Genera nuova popolazione
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_k)
            parent2 = tournament_selection(population, fitnesses, tournament_k)
            if np.array_equal(parent1, parent2):
                parent2 = tournament_selection(population, fitnesses, tournament_k)

            # Crossover
            if random.random() < crossover_rate:
                cx = random.choice(crossover_ops)
                child1, child2 = cx(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Mutazione
            if random.random() < mutation_rate:
                child1 = random.choice(mutation_ops)(child1)
            if random.random() < mutation_rate:
                child2 = random.choice(mutation_ops)(child2)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population
        fitnesses = [tour_length(ind, dist_matrix) for ind in population]

        # Stampa progresso
        if gen % max(1, generations // 10) == 0 or gen == 1:
            print(f"Gen {gen:4d} | Best so far: {best_overall_fit:.4f}")

    return best_overall, best_overall_fit


# ==========================================================
#  ESEMPIO DI ESECUZIONE
# ==========================================================
if __name__ == "__main__":

    best_tour, best_cost = evolutionary_tsp(
        dist_matrix=problem,
        pop_size=50,
        generations=1000,
        crossover_rate=0.95,
        mutation_rate=0.3,
        elitism_count=2,
        tournament_k=3,
        rng_seed=1234,
    )

    print("\n--- RISULTATO FINALE ---")
    print("Miglior costo:", best_cost)
    print("Miglior tour:", best_tour)


Gen    1 | Best so far: -613.2288
Gen  100 | Best so far: -3241.3614
Gen  200 | Best so far: -3685.6032
Gen  300 | Best so far: -3795.8649
Gen  400 | Best so far: -3947.3302
Gen  500 | Best so far: -3966.6927
Gen  600 | Best so far: -3987.5150
Gen  700 | Best so far: -4000.2085
Gen  800 | Best so far: -4000.2085
Gen  900 | Best so far: -4032.3697
Gen 1000 | Best so far: -4112.1754

--- RISULTATO FINALE ---
Miglior costo: -4112.175367079058
Miglior tour: [93 45 25 74 84 56 14  8 64 72  9 27 32 42 60 41 77 48 62 81 61 39 33 99
 11 43 21  1 96 69 35  7 63 13 47 76 65 24 23 67 19  6 52 34 98 22 86 55
 92 75 29 18 80  4 68 88 17 51 66 71 83 59 57 94 53 89 15 85 54 31 73 36
  0  5 30 97 44 90 37 70 91 12 82 95 46 20 79 40 49  3 26 28 16 87 58 78
 10 50  2 38]


##Test Soluzione

In [ ]:
import numpy as np

def check_tsp_solution(tour: np.ndarray, dist_matrix: np.ndarray) -> bool:
    """
    Verifica che una soluzione del TSP rispetti i vincoli fondamentali:
    - Tutte le città sono presenti esattamente una volta.
    - Non ci sono duplicati o omissioni.
    - Le distanze percorse sono finite e non NaN.
    """
    n = dist_matrix.shape[0]
    valid = True

    # --- Controllo 1: dimensione tour ---
    if len(tour) != n:
        print(f"❌ Errore: il tour ha lunghezza {len(tour)}, ma dovrebbero esserci {n} città.")
        valid = False

    # --- Controllo 2: duplicati e omissioni ---
    unique_cities = set(tour)
    if len(unique_cities) != n:
        missing = set(range(n)) - unique_cities
        duplicates = [c for c in tour if list(tour).count(c) > 1]
        print("❌ Errore: il tour non è una permutazione valida delle città.")
        if missing:
            print(f"   → Mancano le città: {sorted(missing)}")
        if duplicates:
            print(f"   → Città duplicate: {sorted(set(duplicates))}")
        valid = False

    # --- Controllo 3: distanze finite ---
    total_length = 0.0
    for i in range(n):
        a, b = tour[i], tour[(i + 1) % n]
        d = dist_matrix[a, b]
        if not np.isfinite(d):
            print(f"❌ Errore: distanza non valida tra città {a} e {b}: {d}")
            valid = False
        total_length += d

    if valid:
        print(f"✅ Tour valido! Lunghezza totale = {total_length:.4f}")
    else:
        print("⚠️ Tour non valido.")
    return valid


# Esempio di utilizzo (dopo aver eseguito evolutionary_tsp)
if __name__ == "__main__":
    # Esempio di matrice di distanze casuale simmetrica
    n_cities = 6
    np.random.seed(0)
    problem = np.random.randint(10, 100, size=(n_cities, n_cities)).astype(float)
    np.fill_diagonal(problem, 0)
    problem = (problem + problem.T) / 2

    # Esegui il TSP evolutivo
    best_tour, best_cost = evolutionary_tsp(problem, generations=200, pop_size=30)

    print("\n--- Verifica soluzione ---")
    check_tsp_solution(best_tour, problem)


Gen    1 | Best so far: 326.5000
Gen   20 | Best so far: 326.5000
Gen   40 | Best so far: 326.5000
Gen   60 | Best so far: 326.5000
Gen   80 | Best so far: 326.5000
Gen  100 | Best so far: 326.5000
Gen  120 | Best so far: 326.5000
Gen  140 | Best so far: 326.5000
Gen  160 | Best so far: 326.5000
Gen  180 | Best so far: 326.5000
Gen  200 | Best so far: 326.5000

--- Verifica soluzione ---
✅ Tour valido! Lunghezza totale = 326.5000
